In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
sst_col_list = [c for c in add_train.columns if 'sst' in c]
sst_col_list

['sst-2010-1',
 'sst-2010-2',
 'sst-2010-3',
 'sst-2010-4',
 'sst-2010-5',
 'sst-2010-6',
 'sst-2010-7',
 'sst-2010-8',
 'sst-2010-9',
 'sst-2010-10']

In [4]:
train_df = pd.read_csv('./fe_v2_train_076.csv')
test_df = pd.read_csv('./fe_v2_test_076.csv')

In [5]:
exclude_cols = ['index', 'startdate']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]

In [6]:
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 129


In [7]:
group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']

In [8]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5']

In [9]:
print("Len:", len(best_cols))

Len: 116


In [10]:
def assign_train_test_df(df, my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()
    return df

def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

def create_feature(df, alias_name, col_name, suffix_num, season_group_cols):
    first_df = create_df_by_group_and_agg(df, season_group_cols, "first", col_name, f"first_{alias_name}_{suffix_num}")
    min_df = create_df_by_group_and_agg(df, season_group_cols, "min", col_name, f"min_{alias_name}_{suffix_num}")
    max_df = create_df_by_group_and_agg(df, season_group_cols, "max", col_name, f"max_{alias_name}_{suffix_num}")
    
    df = df.merge(first_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_first'] = df[col_name] - df[f"first_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_first')
    
    df = df.merge(min_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_min'] = df[col_name] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_min')
    
    df = df.merge(max_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_max'] = df[col_name] - df[f"max_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_max')
    
    df[f'range_{alias_name}_{suffix_num}'] = df[f"max_{alias_name}_{suffix_num}"] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'range_{alias_name}_{suffix_num}')
    
    df[f'scale_{alias_name}_{suffix_num}'] = (df[col_name] - df[f"min_{alias_name}_{suffix_num}"]) / df[f'range_{alias_name}_{suffix_num}']
    df = assign_train_test_df(df, f'scale_{alias_name}_{suffix_num}')
    
    return [df, f'diff_{alias_name}_{suffix_num}_first',
   f'diff_{alias_name}_{suffix_num}_min',
    f'diff_{alias_name}_{suffix_num}_max',
    f'range_{alias_name}_{suffix_num}', f'scale_{alias_name}_{suffix_num}']

In [11]:
slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
wind_col_10 = 'contest-wind-h10-14d__wind-hgt-10'
sst_1_col = 'sst-2010-1'
sst_5_col = 'sst-2010-5'
sst_3_col = 'sst-2010-3'
sst_4_col = 'sst-2010-4'
sst_6_col = 'sst-2010-6'
sst_7_col = 'sst-2010-7'
sst_8_col = 'sst-2010-8'
sst_9_col = 'sst-2010-9'
sst_10_col = 'sst-2010-10'

In [12]:
train_df[sst_col_list] = add_train[sst_col_list]
test_df[sst_col_list] = add_test[sst_col_list]

In [13]:
pevpr_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
train_df[pevpr_col] = add_train[pevpr_col]
test_df[pevpr_col] = add_test[pevpr_col]

In [14]:
df = pd.concat([train_df[main_attrs + [pevpr_col]], test_df[main_attrs + [pevpr_col]]], axis=0)
df.shape

(407088, 130)

In [15]:
pres_res = create_feature(df, 'pres', pres_col, 1, season_group_cols)
df = pres_res[0]
pres_feature = pres_res[1:]

Nan
Nan
Nan


In [16]:
slp_res = create_feature(df, 'slp', slp_col, 1, season_group_cols)
df = slp_res[0]
slp_feature = slp_res[1:]

Nan
Nan
Nan


In [17]:
precip_res = create_feature(df, 'precip', precip_col, 1, season_group_cols)
df = precip_res[0]
precip_feature = precip_res[1:]

Nan
Nan
Nan


In [18]:
sst_1_month_res = create_feature(df, 'sst_1_month', sst_1_col, 1, group_cols)
df = sst_1_month_res[0]
sst_1_month_feature = sst_1_month_res[1:]

Nan
Nan
Nan


In [19]:
sst_5_month_res = create_feature(df, 'sst_5_month', sst_5_col, 1, group_cols)
df = sst_5_month_res[0]
sst_5_month_feature = sst_5_month_res[1:]

Nan
Nan
Nan


In [20]:
sst_3_month_res = create_feature(df, 'sst_3_month', sst_3_col, 1, group_cols)
df = sst_3_month_res[0]
sst_3_month_feature = sst_3_month_res[1:]

Nan
Nan
Nan


In [21]:
sst_4_month_res = create_feature(df, 'sst_4_month', sst_4_col, 1, group_cols)
df = sst_4_month_res[0]
sst_4_month_feature = sst_4_month_res[1:]

Nan
Nan
Nan


In [22]:
sst_6_month_res = create_feature(df, 'sst_6_month', sst_6_col, 1, group_cols)
df = sst_6_month_res[0]
sst_6_month_feature = sst_6_month_res[1:]

Nan
Nan
Nan


In [23]:
wind_10_res = create_feature(df, 'wind_10', wind_col_10, 1, season_group_cols)
df = wind_10_res[0]
wind_10_feature = wind_10_res[1:]

Nan
Nan
Nan


In [24]:
wind_10_res_month = create_feature(df, 'wind_10_month', wind_col_10, 1, group_cols)
df = wind_10_res_month[0]
wind_10_month_feature = wind_10_res_month[1:]

Nan
Nan
Nan


In [25]:
ccsm3_res_month_res = create_feature(df, 'ccsm3_month', ccsm3_col, 1, group_cols)
df = ccsm3_res_month_res[0]
ccsm3_month_feature = ccsm3_res_month_res[1:]

Nan
Nan
Nan


In [26]:
modified_best_cols = [c for c in best_cols]

In [27]:
df['sst_1_lag_1'] = df.groupby(group_cols)[sst_1_col].shift(1)
df = assign_train_test_df(df, 'sst_1_lag_1')

df['sst_1_lag_2'] = df.groupby(group_cols)[sst_1_col].shift(2)
df = assign_train_test_df(df, 'sst_1_lag_2')

df['sst_1_lag_3'] = df.groupby(group_cols)[sst_1_col].shift(3)
df = assign_train_test_df(df, 'sst_1_lag_3')

df['sst_5_lag_1'] = df.groupby(group_cols)[sst_5_col].shift(1)
df = assign_train_test_df(df, 'sst_5_lag_1')

df['sst_3_lag_1'] = df.groupby(group_cols)[sst_3_col].shift(1)
df = assign_train_test_df(df, 'sst_3_lag_1')

df['sst_4_lag_1'] = df.groupby(group_cols)[sst_4_col].shift(1)
df = assign_train_test_df(df, 'sst_4_lag_1')

df['sst_6_lag_1'] = df.groupby(group_cols)[sst_6_col].shift(1)
df = assign_train_test_df(df, 'sst_6_lag_1')


/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [28]:
df['sst_1_lag_1_season'] = df.groupby(season_group_cols)[sst_1_col].shift(1)
df = assign_train_test_df(df, 'sst_1_lag_1_season')
df['sst_5_lag_1_season'] = df.groupby(season_group_cols)[sst_5_col].shift(1)
df = assign_train_test_df(df, 'sst_5_lag_1_season')

df['sst_4_lag_1_season'] = df.groupby(season_group_cols)[sst_4_col].shift(1)
df = assign_train_test_df(df, 'sst_4_lag_1_season')

/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """Entry point for launching an IPython kernel.
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fr

In [29]:
# shift(lag_days).rolling(lag_days).mean()

In [30]:
# df['sst_1_rolling_1'] = df.groupby(group_cols)[sst_1_col].shift(1).rolling(1).mean()
# df = assign_train_test_df(df, 'sst_1_rolling_1')

df['sst_5_rolling_1'] = df.groupby(group_cols)[sst_5_col].shift(1).rolling(1).mean()
df = assign_train_test_df(df, 'sst_5_rolling_1')

/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


In [31]:

pevpr_res = create_feature(df, 'pevpr_col', pevpr_col, 1, group_cols)
df = pevpr_res[0]
pevpr_col_feature = pevpr_res[1:]

Nan
Nan
Nan


/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [32]:
df['pevpr_lag_1'] = df.groupby(group_cols)[pevpr_col].shift(1)
df = assign_train_test_df(df, 'pevpr_lag_1')

/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """Entry point for launching an IPython kernel.


In [33]:
features = modified_best_cols + pres_feature  + ccsm3_month_feature + sst_1_month_feature + ['sst_1_lag_1', 'sst_5_lag_1', 'sst_4_lag_1', 'sst_1_lag_1_season', 'sst_5_lag_1_season', 
                                                                     'sst_4_lag_1_season']

In [34]:
features

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__c

In [35]:
test_df['index'] = add_test['index']

In [36]:
train_df.to_csv('best_train.csv', index=False)
test_df.to_csv('best_test.csv', index=False)

In [37]:
features

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__c

In [ ]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index]) 
    
   
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        
        clf = CatBoostRegressor(iterations=500000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=1000, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_fe_v9.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:30,  6.16s/it]


(375734, 137) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 2243.5625 Total: 24268.3125


Learning rate set to 0.019567
0:	learn: 9.7071550	test: 9.6853663	best: 9.6853663 (0)	total: 117ms	remaining: 16h 17m 2s
1000:	learn: 1.0474364	test: 1.0550155	best: 1.0550155 (1000)	total: 48.7s	remaining: 6h 44m 26s
2000:	learn: 0.8029100	test: 0.8137739	best: 0.8137739 (2000)	total: 1m 39s	remaining: 6h 53m 40s
3000:	learn: 0.6792555	test: 0.6929631	best: 0.6929631 (3000)	total: 2m 5s	remaining: 5h 47m 28s
